<a href="https://colab.research.google.com/github/ishwarraja/SOAI-ERAV2/blob/main/S8/S8_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Define transformations to apply to the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])



In [4]:
# Load the CIFAR10 train dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

# Load the CIFAR10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:01<00:00, 102822650.98it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# Define the class labels
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:


class GroupNormNet(nn.Module):
    def __init__(self):
        super(GroupNormNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv5 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv6 = nn.Conv2d(512, 256, 3, padding=1)
        self.conv7 = nn.Conv2d(256, 128, 3, padding=1)
        self.conv8 = nn.Conv2d(128, 64, 3, padding=1)
        self.conv9 = nn.Conv2d(64, 10, 3, padding=1)
        self.gap = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool(F.relu(self.conv7(x)))
        x = F.relu(self.conv8(x))
        x = self.conv9(x)
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)


In [8]:

class LayerNormNet(nn.Module):
    def __init__(self):
        super(LayerNormNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)

        self.conv7 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv9 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

        self.fc10 = nn.Linear(512, 10)

        self.relu = nn.ReLU()
        self.layernorm = nn.LayerNorm([64, 32, 32])

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pool(x)

        x = self.relu(self.conv4(x))
        x = self.relu(self.conv5(x))
        x = self.relu(self.conv6(x))
        x = self.pool(x)

        x = self.relu(self.conv7(x))
        x = self.relu(self.conv8(x))
        x = self.relu(self.conv9(x))
        x = self.pool(x)

        x = self.global_avg_pool(x)
        x = x.view(-1, 512)
        x = self.fc10(x)

        return x


In [9]:

'''
# Define the network architectures

# Network with Group Normalization
class GroupNormNet(nn.Module):
    # Architecture definition here...

# Network with Layer Normalization
class LayerNormNet(nn.Module):
    # Architecture definition here...

# Network with Batch Normalization
class BatchNormNet(nn.Module):
    # Architecture definition here...


'''

class BatchNormNet(nn.Module):
    def __init__(self):
        super(BatchNormNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(256)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(512)

        self.conv7 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(512)
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn8 = nn.BatchNorm2d(512)
        self.conv9 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn9 = nn.BatchNorm2d(512)

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

        self.fc10 = nn.Linear(512, 10)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)

        x = self.relu(self.bn4(self.conv4(x)))
        x = self.relu(self.bn5(self.conv5(x)))
        x = self.relu(self.bn6(self.conv6(x)))
        x = self.pool(x)

        x = self.relu(self.bn7(self.conv7(x)))
        x = self.relu(self.bn8(self.conv8(x)))
        x = self.relu(self.bn9(self.conv9(x)))
        x = self.pool(x)

        x = self.global_avg_pool(x)
        x = x.view(-1, 512)
        x = self.fc10(x)

        return x


In [7]:



# Training parameters
max_epochs = 20

# Training loop for Group Normalization Network
model = GroupNormNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(max_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100.0 * correct / total

    # Evaluate on test set
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_accuracy = 100.0 * correct / total

    print(f"Epoch [{epoch+1}/{max_epochs}]: Train Loss: {train_loss:.3f}, Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

# Training loop for Layer Normalization Network
# Same code structure as above, replacing the model definition with LayerNormNet and updating the print statements accordingly.

# Training loop for Batch Normalization Network
# Same code structure as above, replacing the model definition with BatchNormNet and updating the print statements accordingly.



Epoch [1/20]: Train Loss: 3599.210, Train Accuracy: 9.77%, Test Accuracy: 10.00%
Epoch [2/20]: Train Loss: 3599.124, Train Accuracy: 9.94%, Test Accuracy: 10.00%
Epoch [3/20]: Train Loss: 3599.129, Train Accuracy: 9.64%, Test Accuracy: 10.00%
Epoch [4/20]: Train Loss: 3599.124, Train Accuracy: 9.81%, Test Accuracy: 10.00%
Epoch [5/20]: Train Loss: 3599.110, Train Accuracy: 9.98%, Test Accuracy: 10.00%
Epoch [6/20]: Train Loss: 3599.123, Train Accuracy: 9.78%, Test Accuracy: 10.00%
Epoch [7/20]: Train Loss: 3599.100, Train Accuracy: 9.91%, Test Accuracy: 10.00%
Epoch [8/20]: Train Loss: 3599.129, Train Accuracy: 9.77%, Test Accuracy: 14.54%
Epoch [9/20]: Train Loss: 3599.120, Train Accuracy: 9.89%, Test Accuracy: 10.00%
Epoch [10/20]: Train Loss: 3599.106, Train Accuracy: 9.76%, Test Accuracy: 10.00%
Epoch [11/20]: Train Loss: 3599.080, Train Accuracy: 9.71%, Test Accuracy: 10.00%
Epoch [12/20]: Train Loss: 3599.127, Train Accuracy: 9.74%, Test Accuracy: 10.00%
Epoch [13/20]: Train Loss

In [25]:



# Define the architectures and load the trained models
group_norm_model = GroupNormNet().to(device)
group_norm_model.load_state_dict(torch.load('/content/test/group_norm_model.txt'))

layer_norm_model = LayerNormNet().to(device)
layer_norm_model.load_state_dict(torch.load('/content/test/layer_norm_model.txt'))

batch_norm_model = BatchNormNet().to(device)
batch_norm_model.load_state_dict(torch.load('/content/test/batch_norm_model.txt'))

# Evaluation function
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100.0 * correct / total
    return accuracy

# Evaluate the models on the test dataset and record the accuracy
group_norm_accuracy = evaluate_model(group_norm_model)
layer_norm_accuracy = evaluate_model(layer_norm_model)
batch_norm_accuracy = evaluate_model(batch_norm_model)

print("Accuracy on the test dataset:")
print("Group Normalization: {:.2f}%".format(group_norm_accuracy))
print("Layer Normalization: {:.2f}%".format(layer_norm_accuracy))
print("Batch Normalization: {:.2f}%".format(batch_norm_accuracy))

# Find misclassified images for Batch Normalization model
batch_norm_model.eval()
misclassified_images = []
misclassified_labels = []
correct_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = batch_norm_model(inputs)
        _, predicted = outputs.max(1)
        misclassified_mask = predicted.ne(labels)  # Mask for misclassified images
        misclassified_images.extend(inputs[misclassified_mask].cpu())
        misclassified_labels.extend(predicted[misclassified_mask].cpu())
        correct_labels.extend(labels[misclassified_mask].cpu())

# Display misclassified images
num_samples = 10
misclassified_images = misclassified_images[:num_samples]
misclassified_labels = misclassified_labels[:num_samples]
correct_labels = correct_labels[:num_samples]

fig, axs = plt.subplots(5, 2, figsize=(10, 15))
fig.suptitle("Misclassified Images (Batch Normalization Model)")

for i in range(num_samples):
    image = misclassified_images[i].permute(1, 2, 0)
    image = (image * 0.5) + 0.5  # Denormalize
    label_pred = misclassified_labels[i].item()
    label_true = correct_labels[i].item()
    axs[i // 2, i % 2].imshow(image)
    axs[i // 2, i % 2].set_title(f"Predicted: {label_pred}, True: {label_true}")
    axs[i // 2, i % 2].axis("off")

plt.tight_layout()
plt.show()


EOFError: ignored

In [15]:
mkdir test
!cd /content/test


/content/test


In [22]:
!touch group_norm_model.txt layer_norm_model.txt group_norm_model.txt

In [26]:
!ls -lt

total 0
-rw-r--r-- 1 root root 0 Jun 21 16:30 group_norm_model.txt
-rw-r--r-- 1 root root 0 Jun 21 16:30 layer_norm_model.txt
-rw-r--r-- 1 root root 0 Jun 21 16:30 batch_norm_model.txt
